In [2]:
import requests
import textract
import json
import time
import pandas as pd
import numpy as np
from crossref.restful import Members
members = Members()
from crossref.restful import Works
works = Works() #could you just alias instead? 
from sklearn.utils import shuffle

API key for Crossref - 1fd3ca24-3bcec314-2e981223-6589eaa9

Ok, first, build a tool that pops through crossref and get metadata (including DOIs) for a whole set of search terms and for a specific publisher name. 

Need to add a '&mailto=jononorato@gmail.com' as an end to the URL request to get us into the "polite" pool.

In [99]:
#new_meta_search(['Synthesis', 'Chemistry', 'Organic'])

Before the url request
653392
After the URL request
Percent complete: 0.15304748145064526
AoJ49+OI9O4CPwRodHRwOi8vZHguZG9pLm9yZy8xMC4xMDAyL2poZXQuMzQzMQ==
https://api.crossref.org/works?filter=member%3A311&query=%5B%27Synthesis%27%2C+%27Chemistry%27%2C+%27Organic%27%5D&select=DOI,link&rows=1000&mailto=jononorato@gmail.com&cursor=AoJ49%2BOI9O4CPwRodHRwOi8vZHguZG9pLm9yZy8xMC4xMDAyL2poZXQuMzQzMQ==
Successful request from the server!
Percent complete: 0.3060949629012905
AoJ1ycLyte8CPwFodHRwOi8vZHguZG9pLm9yZy8xMC4xMDAyL2NwbmMuMg==
https://api.crossref.org/works?filter=member%3A311&query=%5B%27Synthesis%27%2C+%27Chemistry%27%2C+%27Organic%27%5D&select=DOI,link&rows=1000&mailto=jononorato@gmail.com&cursor=AoJ1ycLyte8CPwFodHRwOi8vZHguZG9pLm9yZy8xMC4xMDAyL2NwbmMuMg==
Successful request from the server!
Percent complete: 0.45914244435193574
AoJxq4zrv+8CPwlodHRwOi8vZHguZG9pLm9yZy8xMC4xMDAyL2Fqb2MuMjAxODAwNDMy
https://api.crossref.org/works?filter=member%3A311&query=%5B%27Synthesis%27%2C+%27Chemis

,URL,DOI
0,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/ange.200353477
1,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/anie.200353477
2,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/mrc.1364
3,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/chin.200412185
4,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/chin.200412196
...,...,...
593233,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1890/1051-0761(2001)011[0371:nppitf]2.0.co;2
593234,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/etc.5620160103
593235,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1111/j.1365-2958.2006.05285.x
593236,https://api.wiley.com/onlinelibrary/tdm/v1/art...,10.1002/chin.199147311


In [98]:
def new_meta_search(search_terms, publisher_name = 'Wiley'):
    """"""
    chem_subset = works.query(search_terms)
    pub = next(iter(members.query(publisher_name)))
    pub_id = pub['id']
    chem_subset = chem_subset.filter(member = pub_id)
    doc_number = chem_subset.count()
    print("Before the url request")
    headers = {'CR-Clickthrough-Client-Token': "1fd3ca24-3bcec314-2e981223-6589eaa9", 
               "User-Agent": "Jon Onorato, mailto:jononorato@gmail.com",
               "Connection": 'close'}
    url= chem_subset.url + "&select=DOI,link&rows=1000&mailto=jononorato@gmail.com&cursor="
    #This is our starting cursor value. It will be updated in the loop with each request we make.
    #Standard request will look like:
    #response = requests.get(url+cursor).json()
    cursor = '*'
    url_list = []
    doi_list = []
    saved_docs = 0
    #Build a dataframe to save everything in. 
    full_publist = pd.DataFrame()
    print(doc_number)

    
    #We'll need to make the first request before we get into the while loop for the first time.
    response = requests.get(url+cursor, headers = headers).json()
    print("After the URL request")
    
    while len(response['message']['items']) > 0:
        #Add the total number of papers from the response to the saved docs list. 
        saved_docs += len(response['message']['items'])
        print("Percent complete: " + str(100*saved_docs/doc_number))
        
        #Ok, for every entry in the response, loop through each entry. 
        for entries in response['message']['items']:
            #Check to see if the response item has a link in it
            keycheck = True
            try:
                entries['link']
            except KeyError:
                keycheck = False
            if keycheck:
                #If the link exists, then append stuff
                URL = entries['link'][0]['URL']
                DOI = entries['DOI']
                #Update all http to https
                if URL[:5] != "https" and URL[:4] == 'http':
                    URL = 'https' + URL[4:]
                #Check to see if the URL format is correct. If it is, add it, otherwise you don't add it to the list.
                if URL[8] == 'a':
                    url_list.append(URL)
                    doi_list.append(DOI)
        
        #Build a dataframe of URLs and DOIs from our requests    
        full_publist = pd.DataFrame()
        full_publist['URL'] = url_list
        full_publist['DOI'] = doi_list
        full_publist.to_json(r'/Users/Jonathan/Desktop/main_wiley_fulltext.json')
        
        #Ok, no we need to update the cursor, and make a new request for a new response. 
        cursor = response['message']['next-cursor']
        print(cursor)
        cursor = cursor.replace("+", "%2B")
        print(url+cursor)
        #time.sleep(180)
        response_nojson = requests.get(url+cursor, headers = headers)
        #NEED TO CHECK THE STATUS CODE FROM THIS RESPONSE.
        if response_nojson.status_code == 200:
            print("Successful request from the server!") 
            response = response_nojson.json()
        else:
            print("Status code is bad! The response code is: " + str(response_nojson.status_code))
            break
                
    #Build a dataframe of URLs and DOIs from our requests    
    full_publist['URL'] = url_list
    full_publist['DOI'] = doi_list
        
    #Save the full list somewhere on computer, so we have a master copy somewhere.
    #First make sure there are no duplicates.
    full_publist.drop_duplicates()
    full_publist.to_json(r'/Users/Jonathan/Desktop/main_wiley_fulltext.json')

    
    return full_publist

In [4]:
def article_meta_search(search_terms, publisher_name = 'Wiley', restart = False, current_data = None):
    """A method to collect the metadata from crossref based on specific search terms passed in. The results
    are filtered down to a specific publisher. It then saves all the URLs as a single list."""
    #First, we query crossref, and pull the metadata for specific search terms, and filter by publishers.
    chem_subset = works.query(search_terms)
    pub = next(iter(members.query(publisher_name)))
    pub_id = pub['id']
    chem_subset = chem_subset.filter(member = pub_id)  
    doc_length = chem_subset.count()
    
    #Next build a list of all the URLs, trying to fix all the URL formats if possible. 
    if restart:
        url_list = current_data['URL']
        DOI_list = current_data['DOI']
    else:
        url_list = []
        DOI_list = []
    
    saved_docs = 0
    for items in chem_subset:
        #We need some way to check if this thing has a key that says "link"
        #There are things published by Wiley that don't have links, like book chapters.
        keycheck = True
        saved_docs += 1
        if saved_docs % 1000 == 0:
            print("Current progress: " + str(100.0*saved_docs/doc_length))
            if saved_docs % 10000 == 0:
                full_pub_list = pd.DataFrame([url_list, DOI_list], columns = ["URL", "DOI"])
                with open('/Users/Jonathan/Desktop/temp_wiley_list.json','w') as file:
                    json.dump(full_pub_list, file)
        try:
            items['link']
        except KeyError:
            keycheck = False
        if keycheck:
            URL = items['link'][0]['URL']
            DOI = items['DOI']
            #Update all http to https
            if URL[:5] != "https" and URL[:4] == 'http':
                URL = 'https' + URL[4:]
            #Check to see if the URL format is correct. If it is, add it, otherwise you don't add it to the list.
            if URL[8] == 'a':
                url_list.append(URL)
                DOI_list.append(DOI)
    #Make the two lists into a dataframe
    full_pub_list = pd.DataFrame([url_list, DOI_list], columns = ["URL", "DOI"])
    #Save the full list somewhere on computer, so we have a master copy somewhere.
    with open('/Users/Jonathan/Desktop/master_wiley_list.json','w') as file:
        json.dump(full_pub_list, file)
    
    return full_pub_list

Ok, once we have a list of Wiley URLs, we need to extract the fulltext URLs. We should plan to save current list and pass a current list. 

In [110]:
def get_fulltext(input_filepath, save_path):
    """"""
    #You should update this token with your own API key from Wiley as well as your own email
    #Make sure to keep the connection - closed bit, otherwise you get rate limited really quickly.
    headers = {'CR-Clickthrough-Client-Token': "98e1aa44-e6ddb038-4667e42f-c66c6f22", 
               "User-Agent": "Jon Onorato, mailto:jononorato@gmail.com",
               "Connection": 'close'}
    
    #Ok, now we need to shuffle everything so we know we don't have bias in our data
    full_df = pd.read_json(input_filepath)
    full_df = shuffle(full_df)
    url_list = full_df['URL'].tolist()
    doi_list = full_df['DOI'].tolist()
    
    try:
        error_df = pd.read_json(save_path+'/new_error_counts.json')
    except ValueError:
        error_df = pd.DataFrame(columns = ['URL', 'DOI', 'Status_Code']) 
    
    #Need a plus one to make sure it points to the first non-filled error spot.
    error_counter = len(error_df.index) +1
    basic_counter = 0
    successful_paper_counter = 0
    
    #Ok, now start looping through the full list of URLs
    while len(url_list) > 0:
        #Pull the first url and doi out of the list.
        url = url_list.pop(0)
        doi = doi_list.pop(0)
        basic_counter += 1
        
        #Query the Wiley database. 
        file = requests.get(url = url, headers = headers, allow_redirects = True)
        #print(file.headers)
        print("Number of papers: " + str(basic_counter) + " with status code: " + str(file.status_code))
        time.sleep(14)
        
        if basic_counter % 25 == 0:
            #Re-save the full blown url_list to a file of some sort.
            current_wiley_list = pd.DataFrame()
            current_wiley_list['URL'] = url_list
            current_wiley_list['DOI'] = doi_list
            current_wiley_list.to_json(save_path+"/new_current_wiley_status.json")
        
        #Check now to see if the request was successful.
        if file.status_code == 200:
            #Save the file directly to the desktop. We need to do this to open it elsewhere I think.
            open('/Users/Jonathan/Desktop/file.pdf', 'wb').write(file.content)
            #Now pull the text out of the pdf file. 
            textract_text = textract.process('/Users/Jonathan/Desktop/file.pdf')
            #Ok, now clean up the extracted text of any newline and return characters.
            string_textract = str(textract_text)
            no_n = string_textract.replace('\\n', '')
            no_r_or_n = no_n.replace('\\r', '')
            
            #And we save the text from the pdf file as a .txt. 
            #First remove the doi's '/' so it doesn't mess with saving. Replace with '_'
            doi_no_dash = doi.replace('/', '_')
            file2 = open(f'{save_path}/{doi_no_dash}.txt', 'w')
            file2.write(no_r_or_n)         
            successful_paper_counter += 1
            print(successful_paper_counter)
            
        #Should check to see the response code is not a 429, which is a too many requests error
        elif file.status_code == 429:
            print("Error! We've made too many Wiley requests!")
            #Add the url we were trying to search for back to the url_list
            url_list.append(url)
            doi_list.append(doi)
            #Re-save the full blown url_list to a file of some sort.
            current_wiley_list = pd.DataFrame()
            current_wiley_list['URL'] = url_list
            current_wiley_list['DOI'] = doi_list
            current_wiley_list.to_json(save_path+"/new_current_wiley_status.json")
            print("The remaining URLs have been saved to savepath. Try again later!")
            break
        
        #If 403 we can't get the article because it is not subscribed to by UW
        #If 404, something was broken in the URL path. 
        #If 500, it's because we went too quickly or had a problem with server connection.
        elif file.status_code == 403 or file.status_code == 404 or file.status_code == 500:
            print("Error! Something was wrong with the request that was made. Status code: ")
            print(file.status_code)
            print(url)
            print(doi)
            if file.status_code == 500:
                #Add the url we were trying to search for back to the url_list
                url_list.append(url)
                doi_list.append(doi)
                #Re-save the full blown url_list to a file of some sort.
                current_wiley_list = pd.DataFrame()
                current_wiley_list['URL'] = url_list
                current_wiley_list['DOI'] = doi_list
                current_wiley_list.to_json(save_path+"/new_current_wiley_status.json")
                break

            print("We'll just skip this url for now, but we'll save these URLs somewhere")
            error_df.loc[error_counter] = [url, doi, file.status_code]
            error_df.to_json(save_path+'/new_error_counts.json')
            error_counter += 1
            
        else:
            print("There was a problem of some sort. Status code is: " + str(file.status_code))
            print(file.status_code)
            print(url)
            print(doi)
            url_list.append(url)
            doi_list.append(doi)
            #Re-save the full blown url_list to a file of some sort.
            current_wiley_list = pd.DataFrame()
            current_wiley_list['URL'] = url_list
            current_wiley_list['DOI'] = doi_list
            current_wiley_list.to_json(save_path+"/new_current_wiley_status.json")
            print("The remaining URLs have been saved to the savepath. Try again later!")     
            break
            
    print("You've analyzed all of your URLs! Good job!")

In [ ]:
get_fulltext("/Users/Jonathan/Desktop/WileySynthCorpus/new_current_wiley_status.json", "/Users/Jonathan/Desktop/WileySynthCorpus")

Number of papers: 1 with status code: 200
1
Number of papers: 2 with status code: 200
2
Number of papers: 3 with status code: 200
